# WRITE WHAT THIS NOTEBOOK IS ABOUT HERE.

## Part 1: Prepare the development environment.

In [1]:
from IPython.core.pylabtools import figsize
import math
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn import linear_model, metrics
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, LogisticRegression, Perceptron
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, classification_report, mean_squared_error, r2_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import SVR
import time
import warnings

## Part 2: Tokenize the datasets.

In [2]:
def token(data):
  model_checkpoint = "facebook/esm2_t12_35M_UR50D"
  from transformers import AutoTokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  y=data.iloc[:, -1]
  print("labels:", y.shape)
# before alignment
  fingerprint = []
  adjacency = []
  sequence = []
  for i in range(len(data)):
    compound, protein, interaction = data.iloc[i, :]

    mol = Chem.AddHs(Chem.MolFromSmiles(compound))
    adjacency.append( np.array(Chem.GetAdjacencyMatrix(mol)).flatten())
    fingerprint.append(np.array([int(i) for i in AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024, useChirality=True).ToBitString()]))
    sequence.append( np.array(tokenizer(protein,padding='max_length')['input_ids']))
# alignments
  adjacency =pd.DataFrame(adjacency)
  adjacency= adjacency.fillna(0)

  sequence =pd.DataFrame(sequence)
  sequence= sequence.fillna(0)
# after alignments, combine different parts of data
  fingerprint=pd.DataFrame(fingerprint)
  X = pd.concat([fingerprint, adjacency,sequence], axis=1)
  print("dataset:", X.shape)
  return X, y

def token_simple(data):
  model_checkpoint = "facebook/esm2_t12_35M_UR50D"
  from transformers import AutoTokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  y=data.iloc[:, -1]
  print("labels:", y.shape)
# before alignment
  fingerprint = []
  sequence = []
  for i in range(len(data)):
    compound, protein, interaction = data.iloc[i, :]

    mol = Chem.AddHs(Chem.MolFromSmiles(compound))
    fingerprint.append(np.array([int(i) for i in AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024, useChirality=True).ToBitString()]))
    sequence.append( np.array(tokenizer(protein,padding='max_length')['input_ids']))
# alignments
  sequence =pd.DataFrame(sequence)
  sequence= sequence.fillna(0)
# after alignments, combine different parts of data
  fingerprint=pd.DataFrame(fingerprint)
  X = pd.concat([fingerprint, sequence], axis=1)
  print("dataset:", X.shape)
  print("Number of positive/negative labels:", y.value_counts())
  return X, y

def token_poly(data):
  y=data.iloc[:, -1]
  print("labels:", y.shape)
  fingerprint = []
  for i in range(len(data)):
    compound, protein, interaction = data.iloc[i, :]
    mol = Chem.AddHs(Chem.MolFromSmiles(compound))
    fingerprint.append(np.array([int(i) for i in AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=64, useChirality=True).ToBitString()]))
# after alignments
  X=pd.DataFrame(fingerprint)
  print("dataset:", X.shape)
  return X, y

### Future Work

In [3]:
# apply different protein tokenization from different models
ESM_models = ['facebook/esm2_t48_15B_UR50D', 'facebook/esm2_t36_3B_UR50D', 'facebook/esm2_t33_650M_UR50D', 'facebook/esm2_t30_150M_UR50D', 'facebook/esm2_t12_35M_UR50D', 'facebook/esm2_t6_8M_UR50D']

# apply different fingerprint algorithms for compound 
fpgen1 = AllChem.GetRDKitFPGenerator()
fpgen2 = AllChem.GetAtomPairGenerator()
fpgen3 = AllChem.GetMorganGenerator(radius=2)

# fairness: with different ratios of positive and negative samples for CPIs prediction.
#dataset_0 = dataset.['interaction' = 0][0:1000]
#dataset_1 = dataset.['interaction' = 1][0:1000]

##  Part 3: Supervised Regression Models Comparison

### 3.1 Regression models

In [4]:
# data processing
data = pd.read_csv('test_EC50.csv', header=None)
X, y = token(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Ridge
reg = linear_model.Ridge(alpha=.1)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
test_rmse_ridge=math.sqrt(mean_squared_error(y_test, y_pred))
test_r2_ridge = r2_score(y_test, y_pred)

# Lasso
clf = linear_model.Lasso(alpha=.1)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
test_rmse_lasso=math.sqrt(mean_squared_error(y_test, y_pred))
test_r2_lasso = r2_score(y_test, y_pred)

# SVM
svm = SVR(kernel='rbf',C=1000, gamma=0.1)
svm.fit(X_train, y_train)
y_test_predicted_svm = svm.predict(X_test)
test_rmse_svm = math.sqrt(mean_squared_error(y_test, y_test_predicted_svm))
test_r2_svm = r2_score(y_test, y_test_predicted_svm)

# Random Forest
rnd_forest_reg = RandomForestRegressor(n_estimators=500,  criterion='squared_error',max_features="auto", verbose=1, max_depth=20, n_jobs=-1)
rnd_forest_reg.fit(X_train, y_train)
y_pred = rnd_forest_reg.predict(X_test)
test_rmse_rf=math.sqrt(mean_squared_error(y_test, y_pred))
test_r2_rf = r2_score(y_test, y_pred)

data = [
        ["Ridge Linear Regression", test_rmse_ridge, test_r2_ridge], 
        ["Lasso Linear Regression", test_rmse_lasso, test_r2_lasso],
        ["Support Vector Machine (Gaussian RBF)", test_rmse_svm, test_r2_svm],
        ["Random Forest", test_rmse_rf, test_r2_rf]
       ]


pd.DataFrame(data, columns=["Model", "RMSE", "R2 Score"])

FileNotFoundError: [Errno 2] No such file or directory: 'test_EC50.csv'

In [ ]:
# data processing
data = pd.read_csv('test_EC50.csv', header=None)
data = data.sample(frac=1)
X, y = token_simple(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Ridge
reg = linear_model.Ridge(alpha=.1)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
test_rmse_ridge=math.sqrt(mean_squared_error(y_test, y_pred))
test_r2_ridge = r2_score(y_test, y_pred)

# Lasso
clf = linear_model.Lasso(alpha=.1)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
test_rmse_lasso=math.sqrt(mean_squared_error(y_test, y_pred))
test_r2_lasso = r2_score(y_test, y_pred)

# SVM
svm = SVR(kernel='rbf',C=1000, gamma=0.1)
svm.fit(X_train, y_train)
y_test_predicted_svm = svm.predict(X_test)
test_rmse_svm = math.sqrt(mean_squared_error(y_test, y_test_predicted_svm))
test_r2_svm = r2_score(y_test, y_test_predicted_svm)

# Random Forest
rnd_forest_reg = RandomForestRegressor(n_estimators=500,  criterion='squared_error',verbose=1, max_depth=20, n_jobs=-1)
rnd_forest_reg.fit(X_train, y_train)
y_pred = rnd_forest_reg.predict(X_test)
test_rmse_rf=math.sqrt(mean_squared_error(y_test, y_pred))
test_r2_rf = r2_score(y_test, y_pred)

# MLP regressor
MLP_reg = MLPRegressor(random_state=1, max_iter=500)
MLP_reg.fit(X_train, y_train)
y_pred = MLP_reg.predict(X_test)
test_rmse_MLP=math.sqrt(mean_squared_error(y_test, y_pred))
test_r2_MLP = r2_score(y_test, y_pred)

data = [
        ["Ridge Linear Regression", test_rmse_ridge, test_r2_ridge], 
        ["Lasso Linear Regression", test_rmse_lasso, test_r2_lasso],
        ["Support Vector Machine (Gaussian RBF)", test_rmse_svm, test_r2_svm],
        ["Random Forest", test_rmse_rf, test_r2_rf],
        ["MLP",test_rmse_MLP, test_r2_MLP]
       ]


pd.DataFrame(data, columns=["Model", "RMSE", "R2 Score"])

### 3.2 Hyperparameter Tuning for SGD

In [ ]:
# %%time
warnings.filterwarnings('ignore')

# The param_grid tells Scikit-Learn to evaluate all combinations of the hyperparameter values
param_grid = {'alpha': [0.1, 0.01, 0.001], 'learning_rate': ["constant", "optimal", "invscaling"], 
              'l1_ratio': [1, 0.2, 0], 'max_iter':[100, 400, 1000],'eta0': [0.01, 0.001, 0.0001], 'penalty':['l2', 'l1', 'elasticnet']}

sgd = SGDRegressor()
sgd_cv = GridSearchCV(sgd, param_grid, cv=2, n_jobs=-1)
sgd_cv.fit(X_train[:1000], y_train[:1000])

params_optimal_sgd = sgd_cv.best_params_

print("Best Score in SGD: %f" % sgd_cv.best_score_)
print("Optimal Hyperparameter Values in SGD: ", params_optimal_sgd)
print("\n")

# test the model
lin_reg_sgd = SGDRegressor(**params_optimal_sgd)
lin_reg_sgd.fit(X_train, y_train)
y_test_predicted = lin_reg_sgd.predict(X_test)

print("Root Mean squared error in SGD: %.2f" %math.sqrt(mean_squared_error(y_test, y_test_predicted)))
print("Coefficient of determination r^2 variance score [1 is perfect prediction] in SGD: %.2f"  % r2_score(y_test, y_test_predicted))

### 3.3 polynomial degree: to find out the coefficiency between binding affinity and drugs, which means some characteristic among drugs may play a significant role on binding affinity no matter what protein is involved.

In [ ]:
# data processing
data = pd.read_csv('test_EC50.csv', header=None)
X, y = token_poly(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# SGD Regression
lin_reg_sgd = SGDRegressor(max_iter=1000, eta0=0.01, penalty="elasticnet", l1_ratio=0.0, alpha=0.01)
lin_reg_sgd.fit(X_train.iloc[:,0:1024], y_train)
y_train_predicted = lin_reg_sgd.predict(X_train.iloc[:,:1024])
print("Root Mean squared error for the coefficiency between binding affinity and drugs in SGD: %.2f" %math.sqrt(mean_squared_error(y_train, y_train_predicted)))

# Add polynomial terms with the feature vector using the sklearn PolynomialFeatures class
poly_features = PolynomialFeatures(2)
X_train_poly = poly_features.fit_transform(X_train.iloc[:,:1024])
print("No. of Original Features: ", X_train.iloc[:,:1024].shape[1])
print("No. of Augmented Features: ", X_train_poly.shape[1])
lin_reg_sgd.fit(X_train_poly, y_train)
y_train_predicted = lin_reg_sgd.predict(X_train_poly)
print("Root Mean squared error for the coefficiency between binding affinity and drugs in polynomial solution: %.2f" %math.sqrt(mean_squared_error(y_train, y_train_predicted)))

### Future Work

In [ ]:
# apply the optimized model on other alternative datasets 
data = pd.read_csv('test_IC50.csv', header=None)
X, y = token(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

data = pd.read_csv('test_Kd.csv', header=None)
X, y = token(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

data = pd.read_csv('test_Ki.csv', header=None)
X, y = token(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#  Part 4: Supervised Classification Models Comparison

### 4.1 data processing

In [ ]:
data = pd.read_csv('test_celegans.csv', header=None)
#data = data.sample(frac=1)
X, y = token_simple(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### 4.2 Classification models

In [ ]:
def evaluate(X_train, X_test, y_train, y_test):
    model_name_list = ['Linear Regression', 'ElasticNet Regression', 'Random Forest', 'Extra Trees', 'SVM',
                       'Gradient Boosted','GaussianNB', 'MultinomialNB','Logistic Regression','Perceptron','Multi-Layer Perceptron']
    
    # Instantiate the models
    model1 = LinearRegression()
    model2 = ElasticNet(alpha=1.0, l1_ratio=0.5)
    model3 = RandomForestRegressor(n_estimators=50)
    model4 = ExtraTreesRegressor(n_estimators=50)
    model5 = SVR(kernel='rbf', degree=3, C=1.0, gamma='auto')
    model6 = GradientBoostingRegressor(n_estimators=20)
    model7 = GaussianNB()
    model8 = MultinomialNB(alpha=0.5)
    model9 = LogisticRegression(solver='saga', random_state=0)
    model10 = Perceptron(penalty='l2', alpha=.1)
    model11 = MLPClassifier(random_state=42, hidden_layer_sizes=(3, ), alpha=0.01, max_iter=200,  activation='logistic')
    
    # Dataframe for results
    results = pd.DataFrame(columns=['rmse', 'r2'], index = model_name_list)
    
    # Train and predict with each model
    for i, model in enumerate([model1, model2, model3, model4, model5, model6,model7, model8, model9, model10, model11]):
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        # Metrics
        rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
        r2 = r2_score(y_test, predictions)
        print(model_name_list[i],rmse,r2)
        # Insert results into the dataframe
        model_name = model_name_list[i]
        results.loc[model_name, :] = [rmse, r2]
    return results

results = evaluate(X_train, X_test, y_train, y_test)
print(results)

In [ ]:
# plot the graph
figsize(12, 8)
matplotlib.rcParams['font.size'] = 16
results = results[1:11]

# root mean squared error
ax = plt.subplot(1, 2, 1)
results.sort_values('rmse', ascending = True).plot.bar(y = 'rmse', color = 'g', ax = ax)
plt.title('Root Mean Squared Error'); plt.ylabel('RMSE');

# R2
ax = plt.subplot(1, 2, 2)
results.sort_values('r2', ascending = True).plot.bar(y = 'r2', color = 'b', ax = ax)
plt.title('R2 score'); plt.ylabel('R2');

plt.tight_layout()

### 4.3 KNN model with hyperparameter tuning and optimal tradeoff

In [ ]:
# KNN
%%time

warnings.filterwarnings('ignore')

# The param_grid tells Scikit-Learn to evaluate all combinations of the hyperparameter values
param_grid = {'n_neighbors': np.arange(1,10), 'p': [1, 2, 10], 'weights': ["uniform", "distance"]}

knn_clf = KNeighborsClassifier()

knn_cv = GridSearchCV(knn_clf, param_grid, scoring='f1', cv=2, verbose=3, n_jobs=-1)
knn_cv.fit(X_train.iloc[0:1000], y_train.iloc[0:1000])


params_optimal_knn = knn_cv.best_params_
print("Best Score: %f" % knn_cv.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal_knn)
print("\n")

knn = KNeighborsClassifier(**params_optimal_knn)
y_test_pred = cross_val_predict(knn, X_test, y_test, cv=2)
print("\n RMSE: ", math.sqrt(mean_squared_error(y_test_pred,y_test)))
print("confusion_matrix:", confusion_matrix(y_test, y_test_pred))
precision = precision_score(y_test, y_test_pred) 
print("Precision = %f" % precision)
recall = recall_score(y_test, y_test_pred)
print("Recall = %f" % recall)
f1 = f1_score(y_test, y_test_pred)
print("F1 Score = %f" % f1)
auc= roc_auc_score(y_test,y_test_pred)
print("AUC:",auc)
y_scores = cross_val_predict(knn, X_test, y_test, method="predict_proba", cv=2)[:, 1]
auc= roc_auc_score(y_test,y_scores)
print("AUC with predict_proba:",auc)

In [ ]:
# select the threshold value that gives the best precision/recall tradeoff for our problem

plt.style.use('ggplot')

# Get the 2nd column of the matrix of predicted probabilities for each data point
# The 2nd column stores the probalities of the positive class
y_scores = cross_val_predict(knn, X_train, y_train, method="predict_proba", cv=2)[:, 1]

precisions, recalls, thresholds = precision_recall_curve(y_train, y_scores)

fig = plt.figure(figsize=(10, 6))
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds): 
    plt.plot(thresholds, precisions[:-1], "b--",  linewidth=8, label="Precision") 
    plt.plot(thresholds, recalls[:-1], "p-",  linewidth=3, label="Recall") 
    plt.xlabel("Threshold")
    plt.legend(loc="lower right")
    plt.title('Precision-Recall Curve')
    #plt.xlim([0, 1])
    plt.ylim([0, 1.1])

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

threshold_optimal = -1
for i in range(len(precisions)):
    if(precisions[i] == recalls[i]):
        threshold_optimal = thresholds[i]

print("Optimal Threshold: ", threshold_optimal)

### Future Work

In [ ]:
# apply the optimized model on other alternative datasets

data = pd.read_csv('test_human.csv', header=None)
X, y = token_simple(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Part 5:  Unsupervised Learning Models for Classification

In [ ]:
# data processing
data = pd.read_csv('test_celegans.csv', header=None)
data = data.sample(frac=1)
X, y = token_simple(data)

In [ ]:
print(y.value_counts())

# Outlier
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=2)
y_pred = clf.fit_predict(X)
print((y_pred==1).sum(),(y_pred==-1).sum())
y_pred_update = np.where(y_pred==1,0,1)
print("LocalOutlierFactor result:", math.sqrt(mean_squared_error(y,y_pred_update)))

# KMeans
from sklearn.cluster import KMeans
y_pred =KMeans(n_clusters=2, random_state=0).fit(X).labels_
print((y_pred==1).sum(),(y_pred==0).sum())
y_pred_update = np.where(y_pred==1,0,1)
print("KMeans result:", math.sqrt(mean_squared_error(y,y_pred_update)))

In [ ]:
# Spectral Clustering
from sklearn.cluster import SpectralClustering
y_pred = SpectralClustering(n_clusters=2, random_state=0,assign_labels= 'cluster_qr', degree=1).fit(X).labels_
print((y_pred==1).sum(),(y_pred==0).sum())
y_pred_update = np.where(y_pred==1,0,1)
print("Spectral Clustering result:", math.sqrt(mean_squared_error(y,y_pred_update)))

# Affinity Propagation
from sklearn.cluster import AffinityPropagation
af = AffinityPropagation(random_state=0).fit(X)
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_
print((labels==1).sum(),(labels==-1).sum())
labels = np.where(labels==1,0,1)
n_clusters_ = len(cluster_centers_indices)
print("Affinity Propagation----Estimated number of clusters: %d" % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels))
print("Completeness: %0.3f" % metrics.completeness_score(y, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels))
print("Adjusted Rand Index: %0.3f" % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f" % metrics.adjusted_mutual_info_score(y, labels))